# <p style="color:dodgerblue">02 Create Lambda and Agents</p>
*With Knowledge Bases for Amazon Bedrock, you can give FMs and agents contextual information from your company’s private data sources for Retrieval Augmented Generation (RAG) to deliver more relevant, accurate, and customized responses*  

- this notebook creates the following:
  - lambdas
  - agents
  - iam
    - roles
    - policies
- includes clean up cells to delete all above  
  
(Kernel 3.11.6 - hh:mm)
<hr style="border:1px dotted; color:floralwhite">

# <span style="color:deeppink">GETTING STARTED</span>
# Requirements for this Lab (macOS)
- *See <span style="color:gold">Appendix</span> at the bottom of this lab to install macOS requirements, windows requirements will be similar, apart from Homebrew.*  
  
- Credentials to the AWS account this notebook executes in is provided by AWS configure
  - You must already have an IAM user with code (Command Line Interface) access and AWS access keys to be able to use these credentials in AWS configure  
    
  - arn:aws:iam::###########:user/simon-davies-cli created for this lab

<hr style="border:1px dotted">
<hr style="border:1px dotted;color:greenyellow">

# <p style="color:greenyellow">Set Up Requirements</p>
- we do these setup cells here because we can then use the vars and clients to clean up resources later without having to run multiple cells if we lose the kernel  
  
-  <span style="color:greenyellow">Please note we use us-west-2, region as Bedrock is limited in other reasons<span>

- vars

In [1]:
import boto3
import json

# verify AWS account and store in myAccountNumber
myAccountNumber = boto3.client("sts").get_caller_identity()["Account"]
print(myAccountNumber)

# region - we use us-west-2 as Bedrock is limited in other reasons
myRegion='us-west-2'

# iam
myRoleLambda1="doit-bedrock-recipe-chatbot-lambda-send-recipe-email-role"
myPolicyLambda1="doit-bedrock-recipe-chatbot-lambda-send-recipe-email-policy"
myRoleLambda1ARN='RETRIEVED FROM ROLE BELOW ONCE CREATED'

myRoleLambda2="doit-bedrock-recipe-chatbot-lambda-order-delivery-role"
myPolicyLambda2="doit-bedrock-recipe-chatbot-lambda-order-delivery-policy"
myRoleLambda2ARN='RETRIEVED FROM ROLE BELOW ONCE CREATED'

# NOTE agent resource role must begin with AmazonBedrockExecutionRoleForAgents
myAgentResourceRole1="AmazonBedrockExecutionRoleForAgents_query-knowledge-base"
myAgentResourcePolicy1="AmazonBedrockExecutionRoleForAgents_query-knowledge-base1-policy"
myAgentResourcePolicy2="AmazonBedrockExecutionRoleForAgents_query-knowledge-base2-policy"
myAgentResourceRole1ARN='RETRIEVED FROM ROLE BELOW ONCE CREATED'

# lambda
myLambda1='doit-bedrock-recipe-chatbot-send-recipe-email-fn'
myLambda2='doit-bedrock-recipe-chatbot-order-delivery-fn'
myLambda1ARN='RETRIEVED FROM LAMBDA BELOW ONCE QUERIED'
myLambda2ARN='RETRIEVED FROM LAMBDA BELOW ONCE QUERIED'

# agent
myAgent1='doit-bedrock-recipe-chatbot-agent'
myAgent1KBid='RETRIEVED FROM AGENT BELOW ONCE QUERIED'
myAgent1Aliasid='RETRIEVED FROM ALIAS BELOW ONCE QUERIED'

# action groups
myActionGroup1='doit-bedrock-recipe-chatbot-send-recipe-email-action'
myActionGroup2='doit-bedrock-recipe-chatbot-send-recipe-order-delivery-action'
myActionGroup3='doit-bedrock-recipe-chatbot-more-info-reqd-action'
myActionGroup1Id='RETRIEVED FROM ACTION GROUP QUERY BELOW ONCE QUERIED'
myActionGroup2Id='RETRIEVED FROM ACTION GROUP QUERY BELOW ONCE QUERIED'
myActionGroup3Id='RETRIEVED FROM ACTION GROUP QUERY BELOW ONCE QUERIED'

# knowledge base
myKB='doit-bedrock-recipe-chatbot-kb'
myKBid='RETRIEVED FROM MODEL QUERY BELOW ONCE QUERIED'

# knowledge base models we will use
myQueryingModel='anthropic.claude-v2:1'
myQueryingModelARN='RETRIEVED FROM MODEL QUERY BELOW ONCE QUERIED'

546709318047


# <p style="color:greenyellow">YOU NEED TO PROVIDE THESE!!!</p>
- an email which you need to verify (aws simple email service - ses - will send you ane mail with a link to verefy it)
  - myEmailIdentitySender_SESSandbox
  - this email will be used as a sender in the aws ses service
- an email which you need to verify (aws simple email service - ses - will send you ane mail with a link to verefy it)
  - myEmailReceiver_SESSandbox
  - this email will be used to send recipes to
- location of the resource files
  - myLocalPathForLambda
- If you used the same email for both, you will need to verify both - you will receive 2 separate emails with verification links

-  <span style="color:greenyellow">REMEBER TO CHANGE THIS EMAIL TO YOURS!<span>

In [2]:
# ses
myEmailIdentitySender_SESSandbox='<YOUR EMAIL>'
myEmailReceiver_SESSandbox='<YOUR EMAIL>'

-  <span style="color:greenyellow">REMEBER TO CHANGE THIS PATH TO THE RESOURCES!<span>
-  <span style="color:greenyellow">IF IN AWS JUPYTER MAKE SURE THE 2ND IS UNCOMMENTED<span>

In [ ]:
# local client path for resources
#myLocalPathForLambda='/Users/simondavies/Documents/GitHub/labs/bedrock/recipe-chatbot/Resources/Lambda/'

# jupypter notebook path if notebook is used in AWS for example
myLocalPathForLambda='/home/ec2-user/SageMaker/labs/bedrock/recipe-chatbot/Resources/Lambda/'

- create required clients

In [3]:
# lambda
lambdac = boto3.client('lambda', region_name=myRegion)

# ses
ses = boto3.client('ses', region_name=myRegion)


# iam
iam = boto3.client('iam', region_name=myRegion)

# bedrock
bedrockKB = boto3.client(service_name='bedrock-agent', region_name=myRegion)
bedrockKBRun = boto3.client(service_name='bedrock-agent-runtime', region_name=myRegion)

# logs (cloudwatch)
logs = boto3.client('logs', region_name=myRegion)

- tags for all services that are created - you can never have too many tags!
  - make sure you have a tagging policy in place

In [ ]:
# define tags added to all services we create
myTags = [
    {"Key": "env", "Value": "non_prod"},
    {"Key": "owner", "Value": "doit_bedrock_lab"},
    {"Key": "project", "Value": "doit_bedrock_recipe_chatbot"},
    {"Key": "author", "Value": "simon"},
]
myTagsDct = {
    "env": "non_prod",
    "owner": "doit_bedrock_lab",
    "project": "doit_bedrock_recipe_chatbot",
    "author": "simon",
}

- get myKBid

In [ ]:
# NOTE this assumes you only have 1 knowledge base!!
# if you have more, change max results and look at them to assign the correct kb id
response=bedrockKB.list_knowledge_bases(
    maxResults=1
)

myKBid=response['knowledgeBaseSummaries'][0]['knowledgeBaseId']
myKBid

<hr style="border:1px dotted;color:greenyellow">
<hr style="border:1px dotted;color:orchid">

# <p style="color:orchid">Create IAM</p>
- roles and policies for the services to interact with other services

### <p style="color:orchid">Lambda 1 IAM - send-recipe-email-fn</p>
- allows lambda to create log group and stream and write logs to cloudwatch
- send an email

In [ ]:
# define policy
policyJson = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "logs:CreateLogGroup",
                "logs:CreateLogStream",
                "logs:PutLogEvents",
            ],
            "Resource": "*",
        },
        {
            "Effect": "Allow",
            "Action": [
                "ses:*"
            ],
            "Resource": "*",
        }
    ],
}

# create policy
policy = iam.create_policy(
    PolicyName=myPolicyLambda1,
    PolicyDocument=json.dumps(policyJson),
    Description="Policy for {} lambda".format(myLambda1),
    Tags=[
        *myTags,
    ],
)

# trust policy for the role
roleTrust = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {"Service": "lambda.amazonaws.com"},
            "Action": "sts:AssumeRole",
        }
    ],
}

# create role
role = iam.create_role(
    RoleName=myRoleLambda1,
    AssumeRolePolicyDocument=json.dumps(roleTrust),
    Description="Role for {} lambda".format(myLambda1),
    Tags=[
        *myTags,
    ],
)

# attach policy to role
response = iam.attach_role_policy(
    RoleName=role["Role"]["RoleName"], PolicyArn=policy["Policy"]["Arn"]
)

myRoleLambda1ARN = role['Role']['Arn']

### <p style="color:orchid">Lambda 2 IAM - order-delivery-fn</p>
- allows lambda to create log group and stream and write logs to cloudwatch
- simulate placing an order with a 3rd party

In [ ]:
# define policy
# add any further policies that might be required to allow the lambda to order from a 3rd party
policyJson = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "logs:CreateLogGroup",
                "logs:CreateLogStream",
                "logs:PutLogEvents",
            ],
            "Resource": "*",
        }
    ],
}

# create policy
policy = iam.create_policy(
    PolicyName=myPolicyLambda2,
    PolicyDocument=json.dumps(policyJson),
    Description="Policy for {} lambda".format(myLambda2),
    Tags=[
        *myTags,
    ],
)

# trust policy for the role
roleTrust = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {"Service": "lambda.amazonaws.com"},
            "Action": "sts:AssumeRole",
        }
    ],
}

# create role
role = iam.create_role(
    RoleName=myRoleLambda2,
    AssumeRolePolicyDocument=json.dumps(roleTrust),
    Description="Role for {} lambda".format(myLambda2),
    Tags=[
        *myTags,
    ],
)

# attach policy to role
response = iam.attach_role_policy(
    RoleName=role["Role"]["RoleName"], PolicyArn=policy["Policy"]["Arn"]
)

myRoleLambda2ARN = role['Role']['Arn']

### <p style="color:orchid">Agent 1 IAM - query-knowledge-base-agent</p>
- allows agent to invoke bedrock knowledge base model

In [ ]:
# define policy
policyJson = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": "bedrock:InvokeModel",
            "Resource": [
                "arn:aws:bedrock:us-west-2::foundation-model/{}".format(myQueryingModel)
            ]
        }
    ]
}

# create policy
policy1 = iam.create_policy(
    PolicyName=myAgentResourcePolicy1,
    PolicyDocument=json.dumps(policyJson),
    Description="Policy for {} agent".format(myAgent1),
    Tags=[
        *myTags,
    ],
)

# define policy
policyJson = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "bedrock:Retrieve"
            ],
            "Resource": [
                "arn:aws:bedrock:us-west-2:{}:knowledge-base/{}".format(myAccountNumber, myKBid)
            ]
        }
    ]
}

# create policy
policy2 = iam.create_policy(
    PolicyName=myAgentResourcePolicy2,
    PolicyDocument=json.dumps(policyJson),
    Description="Policy for {} agent".format(myAgent1),
    Tags=[
        *myTags,
    ],
)

# trust policy for the role
roleTrust = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "bedrock.amazonaws.com"
            },
            "Action": "sts:AssumeRole",
            "Condition": {
                "StringEquals": {
                    "aws:SourceAccount": "{}".format(myAccountNumber)
                },
                "ArnLike": {
                    "aws:SourceArn": "arn:aws:bedrock:{}:{}:agent/*".format(myRegion, myAccountNumber)
                }
            }
        }
    ]
}

# create role
role = iam.create_role(
    RoleName=myAgentResourceRole1,
    AssumeRolePolicyDocument=json.dumps(roleTrust),
    Description="Role for {} agent".format(myAgent1),
    Tags=[
        *myTags,
    ],
)

# attach policy to role
response = iam.attach_role_policy(
    RoleName=role["Role"]["RoleName"], PolicyArn=policy1["Policy"]["Arn"]
)
response = iam.attach_role_policy(
    RoleName=role["Role"]["RoleName"], PolicyArn=policy2["Policy"]["Arn"]
)

myAgentResourceRole1ARN = role['Role']['Arn']

<hr style="border:1px dotted;color:orchid">
<hr style="border:1px dotted;color:goldenrod">

# <span style="color:GoldenRod">Create SES</span>
- verify email identity as sender and receiver
  - you will need to verify this identity - check your emails
  - we need to do this because our ses is only in the sandbox and so has limitations on sender and recipient emails  
#### <span style="color:deeppink">verify your email - check your inbox!</span>

In [ ]:
ses.verify_email_identity(
    EmailAddress=myEmailIdentitySender_SESSandbox
)

ses.verify_email_identity(
    EmailAddress=myEmailReceiver_SESSandbox
)

<hr style="border:1px dotted;color:goldenrod">
<hr style="border:1px dotted;color:LightSkyBlue">

# <span style="color:LightSkyBlue">Create Lambda</span>
- You need to create the zip file from the lambda resource folder as create lambda function requires a zipped file
- You can do this from a terminal window as long as you have cd'ed to the folder that contains the function code
  - Eg from the folder than contains the lambda function code (and all of the libraries if any are required) ...
    - *zip -r ../doit-bedrock-recipe-chatbot-send-recipe-email-fn.zip .*
    - Example, Simons Google Drive folder would be:
    - /Users/simondavies/Library/CloudStorage/GoogleDrive-simon.davies@doit.com/Shared drives/Cloud Engineering/apac/Workshops/AWS Bedrock/Bangkok and Tokyo Offices/Labs/Resources/Lambda/send-recipe-email


### <span style="color:LightSkyBlue">Create Lambda 1 - send-recipe-email-fn</span>
- create a lambda to send an email
- will be used by action group 1

In [ ]:
# define vars
myLambdaZip='{}doit-bedrock-recipe-chatbot-send-recipe-email-fn.zip'.format(myLocalPathForLambda)

# Loads the zip file as binary code. 
with open(myLambdaZip, 'rb') as f: 
    code = f.read()
    
# create lambda
myLambdaFunction = lambdac.create_function(
    FunctionName=myLambda1,
    Runtime='python3.12',
    Role=myRoleLambda1ARN,
    Handler='doit-bedrock-recipe-chatbot-send-recipe-email-fn.lambda_handler',
    Code={'ZipFile':code},
    Description='sends an email from details passed in via the bedrock agent event',
    Timeout=30,
    MemorySize=128,
    Publish=True,
    PackageType='Zip',
    Environment={
        'Variables': {
            'SES_SENDER_EMAIL': myEmailIdentitySender_SESSandbox
        }
    },
    Tags=myTagsDct,
    Architectures=[
        'x86_64',
    ],
    LoggingConfig={
        'LogFormat': 'JSON',
        'ApplicationLogLevel': 'INFO',
        'SystemLogLevel': 'WARN'
    }
)

myLambda1ARN=myLambdaFunction['FunctionArn']

- allow bedrock to invoke this lambda

In [ ]:
lambdac.add_permission(
    FunctionName=myLambda1,
    StatementId='bedrock',
    Action='lambda:InvokeFunction',
    Principal='bedrock.amazonaws.com'
)

- log group for lambda

In [ ]:
logs.create_log_group(
    logGroupName='/aws/lambda/' + myLambda1,
    tags=myTagsDct
)

### <span style="color:LightSkyBlue">Create Lambda 2 - order-delivery-fn</span>
- create a lambda to order ingredients
- will be used by action group 2

In [ ]:
# define vars
myLambdaZip='{}doit-bedrock-recipe-chatbot-order-delivery-fn.zip'.format(myLocalPathForLambda)

# Loads the zip file as binary code. 
with open(myLambdaZip, 'rb') as f: 
    code = f.read()
    
# create lambda
myLambdaFunction = lambdac.create_function(
    FunctionName=myLambda2,
    Runtime='python3.12',
    Role=myRoleLambda2ARN,
    Handler='doit-bedrock-recipe-chatbot-order-delivery-fn.lambda_handler',
    Code={'ZipFile':code},
    Description='orders ingredients from details passed in via the bedrock agent event',
    Timeout=30,
    MemorySize=128,
    Publish=True,
    PackageType='Zip',
    Tags=myTagsDct,
    Architectures=[
        'x86_64',
    ],
    LoggingConfig={
        'LogFormat': 'JSON',
        'ApplicationLogLevel': 'INFO',
        'SystemLogLevel': 'WARN'
    }
)

myLambda2ARN=myLambdaFunction['FunctionArn']

- allow bedrock to invoke this lambda

In [ ]:
lambdac.add_permission(
    FunctionName=myLambda2,
    StatementId='bedrock',
    Action='lambda:InvokeFunction',
    Principal='bedrock.amazonaws.com'
)

- log group for lambda

In [ ]:
logs.create_log_group(
    logGroupName='/aws/lambda/' + myLambda2,
    tags=myTagsDct
)

<hr style="border:1px dotted;color:LightSkyBlue">
<hr style="border:1px dotted;color:DarkSeaGreen">

# <p style="color:DarkSeaGreen">Create Agent 1</p>
- Create the agent
  - https://docs.aws.amazon.com/code-library/latest/ug/python_3_bedrock-agent_code_examples.html
  - https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent/client/create_agent.html

In [ ]:
myAgent = bedrockKB.create_agent(
    agentName=myAgent1,
    agentResourceRoleArn=myAgentResourceRole1ARN,
    description='This agent queries the knowledge base {}'.format(myKB),
    foundationModel=myQueryingModel,
    idleSessionTTLInSeconds=1800,
    instruction='You are a top chef skilled in Thai, Japanese and Italian cuisines. ' \
                'You are friendly and polite, and can respond in English, Thai, Japanese or Italian languages. ' \
                'You help with finding dinner recipes and preperation instructions for hungry people to cook for themselves. ' \
                'You can also offer to email the recipe, or order ingredients for delivery.',
    tags=myTagsDct,
)
myAgent1KBid=myAgent['agent']['agentId']

# see if a version exists, ie this isn't the first one
try:
    myAgentVersion=myAgent['agent']['agentVersion']
except:
    myAgentVersion='DRAFT'
myAgentVersion

- adds the knowledge base

In [ ]:
response = bedrockKB.associate_agent_knowledge_base(
    agentId=myAgent1KBid,
    agentVersion=myAgentVersion,
    description='Use to find recipes for Thai, Japanese or Italian cuisine. ' \
                'If not provided, ask user: Thai, Japanese or Italian. ' \
                'Format response so the recipe can be easily followed. ' \
                'Do not use index to cite content.',
    knowledgeBaseId=myKBid,
    knowledgeBaseState='ENABLED'
)

- add the three action groups to reference the lambdas
  - send the recipe email
  - order the ingredients for delivery
  - ask the user for more detail if the chosen action group requires more detail for the api call

In [ ]:
# doit-bedrock-recipe-chatbot-send-recipe-email-action
# openapi payload
payload = {
    "openapi": "3.0.0",
    "info": {
        "title": "Send Recipe API",
        "version": "1.0.0",
        "description": "API for emailing a recipient the ingredients, and cooking instructions for a recipe",
    },
    "paths": {
        "/email": {
            "post": {
                "description": "Send an email providing the ingredients, and cooking instructions for a recipe",
                "operationId": "sendEmail",
                # agent uses the following fields to determine the information it must get from the end user to
                # perform the action group's requirements
                # "parameters": [
                #    {
                #        "name": "emailAddress",
                #        "description": "email addresses of the recipient to send the email to",
                #        "required": True,
                #        "schema": {"type": "string", "format": "email"},
                #    },
                #    {
                #        "name": "firstname",
                #        "description": "first name of the recipient to send the email to",
                #        "required": True,
                #        "schema": {"type": "string"},
                #    },
                #    {
                #        "name": "ingredients",
                #        "description": "list of ingredients",
                #        "required": True,
                #        "schema": {"type": "markdown"},
                #    },
                #    {
                #        "name": "instructions",
                #        "description": "cooking instructions",
                #        "required": True,
                #        "schema": {"type": "markdown"},
                #    },
                # ],
                "requestBody": {
                    "required": True,
                    "content": {
                        "application/json": {
                            "schema": {
                                "type": "object",
                                "properties": {
                                    "emailAddress": {
                                        "type": "string",
                                        "format": "email",
                                        "description": "email addresses of the recipient to send the email to",
                                    },
                                    "firstname": {
                                        "type": "string",
                                        "description": "first name of the recipient to send the email to",
                                    },
                                    "ingredients": {
                                        "type": "markdown",
                                        "description": "list of ingredients",
                                    },
                                    "instructions": {
                                        "type": "markdown",
                                        "description": "cooking instructions",
                                    },
                                },
                                "required": [
                                    "emailAddress",
                                    "firstname",
                                    "ingredients",
                                    "instructions",
                                ],
                            }
                        }
                    },
                },
                "responses": {
                    "200": {
                        "description": "Email sent successfully",
                        "content": {
                            "application/json": {
                                "schema": {
                                    "type": "object",
                                    "properties": {
                                        "sendEmailStatus": {
                                            "type": "string",
                                            "description": "Status of send email",
                                        }
                                    },
                                }
                            }
                        },
                    },
                    "400": {
                        "description": "Bad request. One or more required fields are missing or invalid."
                    },
                },
            }
        }
    },
}

response = bedrockKB.create_agent_action_group(
    actionGroupExecutor={"lambda": myLambda1ARN},
    actionGroupName=myActionGroup1,
    actionGroupState="ENABLED",
    agentId=myAgent1KBid,
    agentVersion=myAgentVersion,
    apiSchema={
        "payload": json.dumps(payload),
    },
    description="Send an email containing the recipe and cooking instructions to a recipient email address",
)
myActionGroup1Id = response["agentActionGroup"]["actionGroupId"]

In [ ]:
# doit-bedrock-recipe-chatbot-send-recipe-order-delivery-action
# openapi payload
payload = {
    "openapi": "3.0.0",
    "info": {
        "title": "Order Ingredients API",
        "version": "1.0.0",
        "description": "API for ordering the recipe ingredients from an online store for immediate delivery",
    },
    "paths": {
        "/order": {
            "post": {
                "description": "Order the ingredients required by a recipe",
                "operationId": "orderIngredients",
                # agent uses the following fields to determine the information it must get from the end user to
                # perform the action group's requirements
                # "parameters": [
                #    {
                #        "name": "deliveryAddress",
                #        "description": "delivery addresses for the order",
                #        "required": True,
                #        "schema": {"type": "string"},
                #    },
                #    {
                #        "name": "firstname",
                #        "description": "first name of the person placing the order",
                #        "required": True,
                #        "schema": {"type": "string"},
                #    },
                #    {
                #        "name": "ingredients",
                #        "description": "list of ingredients to order",
                #        "required": True,
                #        "schema": {"type": "markdown"},
                #    },
                # ],
                "requestBody": {
                    "required": True,
                    "content": {
                        "application/json": {
                            "schema": {
                                "type": "object",
                                "properties": {
                                    "deliveryAddress": {
                                        "type": "string",
                                        "description": "delivery addresses for the order",
                                    },
                                    "firstname": {
                                        "type": "string",
                                        "description": "first name of the person placing the order",
                                    },
                                    "ingredients": {
                                        "type": "markdown",
                                        "description": "list of ingredients to order",
                                    },
                                },
                                "required": [
                                    "deliveryAddress",
                                    "firstname",
                                    "ingredients",
                                ],
                            }
                        }
                    },
                },
                "responses": {
                    "200": {
                        "description": "Order placed successfully",
                        "content": {
                            "application/json": {
                                "schema": {
                                    "type": "object",
                                    "properties": {
                                        "placeOrderStatus": {
                                            "type": "string",
                                            "description": "Status of order",
                                        },
                                        "orderRef": {
                                            "type": "string",
                                            "description": "Order reference",
                                        }
                                    },
                                }
                            }
                        },
                    },
                    "400": {
                        "description": "Bad request. One or more required fields are missing or invalid."
                    },
                },
            }
        }
    },
}

response = bedrockKB.create_agent_action_group(
    actionGroupExecutor={"lambda": myLambda2ARN},
    actionGroupName=myActionGroup2,
    actionGroupState="ENABLED",
    agentId=myAgent1KBid,
    agentVersion=myAgentVersion,
    apiSchema={
        "payload": json.dumps(payload),
    },
    description="Place an order for the ingredients of the recipe to be delivered",
)
myActionGroup2Id = response["agentActionGroup"]["actionGroupId"]

In [ ]:
# doit-bedrock-recipe-chatbot-more-info-reqd-action
# NOTE During orchestration, if your agent determines that it needs to invoke an API in an action group, 
# but doesn’t have enough information to complete the API request, it will invoke this action group instead 
# and return an Observation reprompting the user for more information.
response = bedrockKB.create_agent_action_group(
    actionGroupName=myActionGroup3,
    actionGroupState='ENABLED',
    agentId=myAgent1KBid,
    agentVersion=myAgentVersion,
    parentActionGroupSignature='AMAZON.UserInput'
)
myActionGroup3Id=response["agentActionGroup"]["actionGroupId"]

- prepare the agent for testing

In [ ]:
bedrockKB.prepare_agent(
    agentId=myAgent1KBid
)

- wait until the agent is prepared
#### <span style="color:deeppink">you can run the following cell multiple times until the status is PREPARED</span>

In [ ]:
response = bedrockKB.get_agent(
    agentId=myAgent1KBid
)
response['agent']['agentStatus']

- create an alias and version for production use by your applications

In [ ]:
response = bedrockKB.create_agent_alias(
    agentAliasName='{}-alias'.format(myAgent1),
    agentId=myAgent1KBid,
    description='Release of agent {}'.format(myAgent1),
    tags=myTagsDct
)
myAgent1Aliasid=response['agentAlias']['agentAliasId']

<hr style="border:1px dotted;color:DarkSeaGreen">
<hr style="border:1px dotted;color:deeppink">

# <p style="color:deeppink">STACK 02 COMPLETE!</p>

# <p style="color:deeppink">Example Use of Published Agent</p>
- the following code can be used in your projects to invoke the agent we just created

- create a method we can call to invoke the agent

In [ ]:
from botocore.exceptions import ClientError
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

def invoke_agent(mySession, myPrompt, endConv):
    # we can provide session state if required
    # https://docs.aws.amazon.com/bedrock/latest/userguide/agents-session-state.html

    try:
        response = bedrockKBRun.invoke_agent(
            agentAliasId=myAgent1Aliasid,
            agentId=myAgent1KBid,
            enableTrace=True,
            endSession=endConv,
            inputText=myPrompt,
            sessionId=mySession,
        )

        completion = ""

        for event in response.get("completion"):
            print(event)
            try:
                chunk = event["chunk"]
                completion = completion + chunk["bytes"].decode()
            except:
                # must be a trace
                chunk = ""
            
    except ClientError as e:
        logger.error(f"Couldn't invoke agent. {e}")
        raise

    return completion

- uncomment each prompt in turn to converse with the agent
- try your own
- wait until the cell finishes processing before sending the next prompt! 
- each prompt can take a few seconds, and lots of output is generated and displayed
- you can review each step (pre-processing, orchestration, action groip, knowledge base)
- it has not finished until you see the print RESPONSE: blah blah blah
#### <span style="color:deeppink">change the following in code below:</span>
  - \<YOUR_EMAIL> must be the same as the one you defined above in myEmailReceiver_SESSandbox
    - this is because we are using ses in a sandbox and it can only use verified emails
    - <span style="color:deeppink">NOTE this has already been coded below to use it so no need to change</span>
  - \<your name>
  - \<your address>

If you receieve an error, or don't receive the email, have you verified your email after using it as a verified SES identity above?
  
The code can run for a few seconds. Wait for the RESPONSE to appear in the output before running the next prompt.
  
Check your spam folder for emails.

In [ ]:
#Set endConv parameter to True when you want to end the conversation - it will die after 10 minutes regardless
mySession="doit-bedrock-recipe-chatbot2"
myPrompt="hello, do you have any recipes I could cook for dinner?"
myPrompt="Italian please"
myPrompt="No dietary restrictions, do you have a recipe for Cheese Stuffed Shells with Bolognese Sauce please"
myPrompt="Could you email me that recipe please?"
myPrompt="Yes, send me the email, my email is {YOUR_EMAIL} and my name is Simon".format(YOUR_EMAIL=myEmailReceiver_SESSandbox)
#myPrompt="can you order the ingredients for me"
#myPrompt="my name is <YOUR NAME> and my address is <YOUR ADDRESS>"
#myPrompt="thankyou"

endConv=False
completion=invoke_agent(mySession,myPrompt,endConv)
print("RESPONSE:\n{}".format(completion))

- you should now try your agent via the AWS console
  - look at its different traces
  - the lambda functionality
  - the action groups
  - etc

<hr style="border:1px dotted;color:deeppink">
<hr style="border:1px dotted;color:orangered">

# <p style="color:orangered">Clean Up - DO NOT DO THIS IN THIS LAB!!!!!</p>
# <p style="color:orangered">DO NOT RUN THESE UNLESS YOU WANT TO DESTROY EVERYTHING</p>
- If you have lost the Kernel, run the cells contained in the <span style="color:greenyellow">Set Up Requirements<span> section before the cells below

- NOTE if you have lost the kernel, you will need to manually get the id's

In [ ]:
# ses identity
ses.delete_identity(
    Identity=myEmailIdentitySender_SESSandbox
)

ses.delete_identity(
    Identity=myEmailReceiver_SESSandbox
)

In [ ]:
# agent alias
bedrockKB.delete_agent_alias(
    agentAliasId=myAgent1Aliasid,
    agentId=myAgent1KBid
)


In [ ]:
# delete version
bedrockKB.delete_agent_version(
    agentId=myAgent1KBid,
    agentVersion='1',
    skipResourceInUseCheck=False
)


In [ ]:
# agent action groups
bedrockKB.delete_agent_action_group(
    actionGroupId=myActionGroup1Id,
    agentId=myAgent1KBid,
    agentVersion=myAgentVersion,
    skipResourceInUseCheck=True
)

bedrockKB.delete_agent_action_group(
    actionGroupId=myActionGroup2Id,
    agentId=myAgent1KBid,
    agentVersion=myAgentVersion,
    skipResourceInUseCheck=True
)

bedrockKB.delete_agent_action_group(
    actionGroupId=myActionGroup3Id,
    agentId=myAgent1KBid,
    agentVersion=myAgentVersion,
    skipResourceInUseCheck=True
)

In [ ]:
# diassociates knowledge base from agent
bedrockKB.disassociate_agent_knowledge_base(
    agentId=myAgent1KBid,
    agentVersion=myAgentVersion,
    knowledgeBaseId=myKBid
)

In [ ]:
# delete agent
bedrockKB.delete_agent(
    agentId=myAgent1KBid,
    skipResourceInUseCheck=True
)

In [ ]:
# lambdas
lambdac.delete_function(FunctionName=myLambda1)
lambdac.delete_function(FunctionName=myLambda2)

In [ ]:
# delete iam roles and policies

# lambda 1
iam.detach_role_policy(
    RoleName=myRoleLambda1, PolicyArn='arn:aws:iam::{}:policy/{}'.format(myAccountNumber, myPolicyLambda1)
)

iam.delete_role(RoleName=myRoleLambda1)
iam.delete_policy(PolicyArn='arn:aws:iam::{}:policy/{}'.format(myAccountNumber, myPolicyLambda1))

# lambda 2
iam.detach_role_policy(
    RoleName=myRoleLambda2, PolicyArn='arn:aws:iam::{}:policy/{}'.format(myAccountNumber, myPolicyLambda2)
)

iam.delete_role(RoleName=myRoleLambda2)
iam.delete_policy(PolicyArn='arn:aws:iam::{}:policy/{}'.format(myAccountNumber, myPolicyLambda2))

# agent 1
iam.detach_role_policy(
    RoleName=myAgentResourceRole1, PolicyArn='arn:aws:iam::{}:policy/{}'.format(myAccountNumber, myAgentResourcePolicy1)
)
iam.detach_role_policy(
    RoleName=myAgentResourceRole1, PolicyArn='arn:aws:iam::{}:policy/{}'.format(myAccountNumber, myAgentResourcePolicy2)
)

iam.delete_role(RoleName=myAgentResourceRole1)
iam.delete_policy(PolicyArn='arn:aws:iam::{}:policy/{}'.format(myAccountNumber, myAgentResourcePolicy1))
iam.delete_policy(PolicyArn='arn:aws:iam::{}:policy/{}'.format(myAccountNumber, myAgentResourcePolicy2))

In [ ]:
# log group and log streams
logs.delete_log_group(
    logGroupName='/aws/lambda/' + myLambda1
)

logs.delete_log_group(
    logGroupName='/aws/lambda/' + myLambda2
)

<hr style="border:1px dotted;color:coral">
<hr style="border:1px dotted;color:gold">

# <p style="color:gold">Appendix - Jupyter Install Requirements (macOS)</p>
#### <p style="color:deeppink">- If you are running VSCode on a laptop, follow all of below.<br>- If you are running Jupyter inside an AWS Account, you don't need to do anything!</p>

  - Credentials to the AWS account this notebook executes in is provided by AWS configure
  - You must already have an IAM user with code (Command Line Interface) access and AWS access keys to be able to use these credentials in AWS configure  
    
  - arn:aws:iam::###########:user/simon-davies-cli created for this lab  

### <p style="color:gold">1. Homebrew</p> 
If you haven't installed Homebrew, you can install it by running the following command here or in the terminal:

In [ ]:
%%bash
sudo /bin/bash -c "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/HEAD/install.sh)"

### <p style="color:gold">1.1 Virtual Environments</p> 
- You can create a virtual environment that ensures any libraries you install are restricted to the venv.
  - https://code.visualstudio.com/docs/python/environments
- To enable the virtual environment once you have created it, ensure you open the folder in vs code rather than individual files.

In [ ]:
%%bash
sudo /bin/bash -c "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/HEAD/install.sh)"

### <p style="color:gold">1.2 Python</p> 
Once Homebrew is installed, you can install Python using the following command  
*check what you have before installing/upgrading*  
*you will need to quit and restart vsCode to use python once installed (or updated)*

In [ ]:
%%bash
python3 --version
which python3

In [ ]:
%%bash
brew install python

### <p style="color:gold">2. boto3 and other Python requirements</p> 
* boto3 must be installed on your client
  * *Boto3 is the Amazon Web Services (AWS) Software Development Kit (SDK) for Python, which allows Python developers to write software that makes use of services like Amazon S3 and Amazon EC2.*
  * https://boto3.amazonaws.com/v1/documentation/api/latest/index.html  
  
*check what you have before installing/upgrading*  

In [ ]:
%%bash
python3 -m pip show boto3

In [ ]:
pip install -U boto3

In [ ]:
pip install -U langchain

### <p style="color:gold">3. aws configure</p> 
*Configure aws configure with credentials, and a user that has all of the Bedrock IAM policies required*  
https://docs.aws.amazon.com/bedrock/latest/userguide/security_iam_id-based-policy-examples.html

In [ ]:
%%bash
aws sts get-caller-identity

### <p style="color:gold">4. Request Bedrock model access</p> 
*You must request access to the models required, you may need to provide use case details before you are able to request*  
*Make sure you request in the region you intend to use the models in, this lab is us-west-2*  
https://us-west-2.console.aws.amazon.com/bedrock/home?region=us-west-2#/modelaccess  

Models required in this lab:

* See code above for use of models and what access to request

#### Pricing
*Use 6 characters per token as an approximation for the number of tokens.*  
https://aws.amazon.com/bedrock/pricing/  
https://docs.aws.amazon.com/bedrock/latest/userguide/model-customization-prepare.html

<hr style="border:1px dotted;color:gold">